In [6]:
import numpy as np
import ast
import urllib
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import scipy.optimize
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [7]:
def parseDataFromFile(fname):
    """ Read through the file and return results"""
    for l in open(fname, encoding="utf8"):
        yield ast.literal_eval(l)

In [8]:
dataset = list(parseDataFromFile("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json"))
print(dataset[0])

{'hours': 0.3, 'gameID': 'b96045472', 'hours_transformed': 0.37851162325372983, 'early_access': False, 'date': '2015-04-08', 'text': '+1', 'userID': 'u01561183'}


In [9]:
print(len(dataset))
training_validation_boundary = 165000
# Here, we compute the boundaries between the training and validation set
training_set = dataset[:training_validation_boundary]
validation_set = dataset[training_validation_boundary:]

175000


In [10]:
usersPerGame = defaultdict(set) # list of userIDs per game
gamesPerUser = defaultdict(set) # list of gameIDs per user
reviewsPerUser = defaultdict(list)
reviewsPerGame = defaultdict(list)
gameIDs = [] # This contains the entire list of games

usersPerGame_training = defaultdict(set) # list of userIDs per game
gamesPerUser_training = defaultdict(set) # list of gameIDs per user
reviewsPerUser_training = defaultdict(list)
reviewsPerGame_training = defaultdict(list)
gameIDs_training = [] # This contains the entire list of games

usersPerGame_validation = defaultdict(set) # list of userIDs per game
gamesPerUser_validation = defaultdict(set) # list of gameIDs per user
reviewsPerUser_validation = defaultdict(list)
reviewsPerGame_validation = defaultdict(list)
gameIDs_validation = [] # This contains the entire list of games

# Computing maps and lists for all samples
for d in dataset:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame[gameID].add(userID)
    gamesPerUser[userID].add(gameID)
    reviewsPerUser[userID].append(d)
    reviewsPerGame[gameID].append(d)
    if gameID not in gameIDs:
        gameIDs.append(gameID)
        
# Computing maps and lists for training set samples
for d in training_set:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame_training[gameID].add(userID)
    gamesPerUser_training[userID].add(gameID) # list of gameIDs per user
    reviewsPerUser_training[userID].append(d)
    reviewsPerGame_training[gameID].append(d)
    if gameID not in gameIDs_training:
        gameIDs_training.append(gameID)
        
# Computing maps and lists for training set samples
for d in validation_set:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame_validation[gameID].add(userID)
    gamesPerUser_validation[userID].add(gameID) # list of gameIDs per user
    reviewsPerUser_validation[userID].append(d)
    reviewsPerGame_validation[gameID].append(d)
    if gameID not in gameIDs_training:
        gameIDs_validation.append(gameID)
print("Done...")

Done...


In [ ]:
for d in training_set:
    

In [11]:
def perform_duplication(dataset, start, end):
    assert(start <= len(dataset) and end <= len(dataset))
    new_set = []
    print("Start = {}, end = {}".format(start, end))
    for d in dataset[start:end]:
        user, game = d['userID'], d['gameID']
        gamesNotPlayed = []
        for game in gameIDs:
            if(game not in gamesPerUser[user]):
                gamesNotPlayed.append(game)
                
        random.seed()
        randomGame = random.choice(gamesNotPlayed)
        new_data = {'hours': 0, 'gameID': str(randomGame),\
                'hours_transformed': 0, 'early_access': False,\
                'date': '', 'text': '', 'userID': str(userID)}
        new_set.append(new_data)
    return dataset + new_set

In [41]:
new_training_set = perform_duplication(training_set, 0, len(training_set))
new_validation_set = perform_duplication(validation_set, 0, len(validation_set))
print("Lengths of new datasets = {} and {}".format(len(new_training_set), len(new_validation_set)))

Start = 0, end = 165000
Start = 0, end = 10000
Lengths of new datasets = 330000 and 20000


In [42]:
import math

def jaccard(s1, s2):
    numerator = len(s1.intersection(s2))
    denominator = len(s1.union(s2))
    return numerator/denominator

def cosine(s1, s2):
    numerator = len(s1.intersection(s2))
    denominator = math.sqrt(len(s1) * len(s2))
    return numerator/denominator

def similarity_popularity(dataset, sim_th, pop_th, start, end):
    y_actual, y_predicted = [], []
    idx, numerator = 0, 0
    for d in dataset:
        user, game = d['userID'], d['gameID']
        # Compute game to game similarity
        Users_g = usersPerGame[game]
        maxSim = -1
        for g2 in gamesPerUser[user]:
            if(g2 == game):
                continue
            Users_g2 = usersPerGame[g2]
            sim = jaccard(Users_g, Users_g2)
            maxSim = max(sim, maxSim)
    
        if(maxSim > sim_th or len(reviewsPerGame[game]) > pop_th):
            y_predicted.append(1)
        else:
            y_predicted.append(0)
    
        if(idx >= start and idx < end):
            y_actual.append(0)
        else:
            y_actual.append(1)
        
        if(y_actual[idx] == y_predicted[idx]):
            numerator += 1
            
        idx += 1
    # Compute the ratio of numerator / denominator
    return numerator/len(y_predicted)

def similarity_popularity_cosine(dataset, sim_th, pop_th, start, end):
    y_actual, y_predicted = [], []
    idx, numerator = 0, 0
    for d in dataset:
        user, game = d['userID'], d['gameID']
        # Compute game to game similarity
        Users_g = usersPerGame[game]
        maxSim = -1
        for g2 in gamesPerUser[user]:
            if(g2 == game):
                continue
            Users_g2 = usersPerGame[g2]
            sim = cosine(Users_g, Users_g2)
            maxSim = max(sim, maxSim)
        # 
        if(maxSim > sim_th or len(reviewsPerGame[game]) > pop_th):
            y_predicted.append(1)
        else:
            y_predicted.append(0)
    
        if(idx >= start and idx < end):
            y_actual.append(0)
        else:
            y_actual.append(1)
        
        if(y_actual[idx] == y_predicted[idx]):
            numerator += 1
            
        idx += 1
    # Compute the ratio of numerator / denominator
    return numerator/len(y_predicted)

In [43]:
# sim_thresholds = np.arange(0.1, 0.25, 0.01)
# pop_thresholds = np.arange(20, 120, 20)
# accuracies = defaultdict(dict)
# for sim_th in sim_thresholds:
#     for pop_th in pop_thresholds:
#         acc = similarity_popularity(new_validation_set, sim_th, pop_th,
#                                     len(new_validation_set)//2,len(new_validation_set))
#         accuracies[sim_th][pop_th] = acc
#         print("Accuracy at (x, y) = ({}, {}) is {}".format(sim_th, pop_th, acc))

# accuracy_values = []
# for key, value in accuracies.items():
#     values = []
#     for k2, v2 in value.items():
#         values.append(v2)
#     accuracy_values.append(values)
# print(sim_thresholds)
# print(pop_thresholds)
# print(accuracy_values)
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.contour3D(pop_thresholds, sim_thresholds, accuracy_values, 50, cmap='binary')
# ax.set_xlabel('x')
# ax.set_ylabel('y')
# ax.set_zlabel('z')

In [44]:
test_set = []
prediction_file = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Played_60.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Played.txt"):
    if l.startswith("userID"):
        #header
        prediction_file.write(l)
        continue
    u,g = l.strip().split('-')
    val = {}
    val['userID'], val['gameID'] = u, g
    test_set.append(val)


def similarity_popularity(dataset, sim_th, pop_th):
    idx, numerator = 0, 0
    for d in dataset:
        user, game = d['userID'], d['gameID']
        # Compute game to game similarity
        Users_g = usersPerGame[game]
        maxSim = -1
        for g2 in gamesPerUser[user]:
            if(g2 == game):
                continue
            Users_g2 = usersPerGame[g2]
            sim = jaccard(Users_g, Users_g2)
            maxSim = max(sim, maxSim)
    
        if(maxSim > sim_th or len(reviewsPerGame[game]) > pop_th):
            prediction_file.write(user + '-' + game + ",1\n")
        else:
            prediction_file.write(user + '-' + game + ",0\n")
            
similarity_popularity(test_set, 0.11, 60)
prediction_file.close()

In [45]:
def similarity_popularity_features(dataset):
    idx, numerator = 0, 0
    features = []
#     max_length = -1
#     for game in reviewsPerGame:
#         max_length = max(max_length, len(reviewsPerGame[game]))
#     print("Max length of all games is {}".format(max_length))
    for d in dataset:
        u, g = d['userID'], d['gameID']
        games_u = gamesPerUser[u]
        max_sim = -1
        for u2 in usersPerGame[g]:
            if u2 == u:
                continue
            games_u2 = gamesPerUser[u2]
            sim = jaccard(games_u, games_u2)
            max_sim = max(max_sim, sim)
        # Computing updated features  Similarity between users and len(reviewsPerGame)
        features.append([1, max_sim, len(reviewsPerUser[u])])
    return features

In [46]:
def compute_confusion_matrix(Y_actual, Y_predict):
    """ This function is used to compute the confusion matrix"""
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    assert(len(Y_actual) == len(Y_predict))
    for i in range(len(Y_actual)):
        if(Y_actual[i] == 1 and Y_predict[i] == 1):
            TP += 1
        elif(Y_actual[i] == 0 and Y_predict[i] == 1):
            FP += 1
        elif(Y_actual[i] == 0 and Y_predict[i] == 0):
            TN += 1
        elif(Y_actual[i] == 1 and Y_predict[i] == 0):
            FN += 1
    return (TP, TN, FP, FN)

# Compute the ratios for computing the TP, TN, FP and FN
def compute_rates(TP, TN, FP, FN):
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    BER = 0.5 * (FPR + FNR)
    return (TPR, TNR, FPR, FNR, BER)

In [47]:
X_train = similarity_popularity_features(new_training_set)
Y_train = [1 if i < len(new_training_set)/2 else 0 for i in range(len(new_training_set))]

X_test = similarity_popularity_features(new_validation_set)
Y_test = [1 if i < len(new_validation_set)/2 else 0 for i in range(len(new_validation_set))]

lambdas = [10 ** i for i in np.arange(-5,5,dtype=float)]
accuracies = {}
for lamb in lambdas:
    model = linear_model.LogisticRegression(C=lamb,class_weight='balanced')
    model.fit(X_train, Y_train)
    Y_predictions = model.predict(X_test)

    (TP, TN, FP, FN) = compute_confusion_matrix(Y_test, Y_predictions)
    (TPR, TNR, FPR, FNR, BER) = compute_rates(TP, TN, FP, FN)
    acc = (TP + TN)/(TP + TN + FP + FN)
    print("Accuracy @ lamb = {} is {}".format(lamb, acc))
    print("BER @lamb = {} is {}".format(lamb, BER))
    accuracies[lamb] = acc
print("Accuracies = {}".format(accuracies))

Accuracy @ lamb = 1e-05 is 0.68575
BER @lamb = 1e-05 is 0.31425
Accuracy @ lamb = 0.0001 is 0.6855
BER @lamb = 0.0001 is 0.3145
Accuracy @ lamb = 0.001 is 0.70075
BER @lamb = 0.001 is 0.29925
Accuracy @ lamb = 0.01 is 0.8205
BER @lamb = 0.01 is 0.1795
Accuracy @ lamb = 0.1 is 0.83625
BER @lamb = 0.1 is 0.16375
Accuracy @ lamb = 1.0 is 0.84265
BER @lamb = 1.0 is 0.15735
Accuracy @ lamb = 10.0 is 0.8442
BER @lamb = 10.0 is 0.1558
Accuracy @ lamb = 100.0 is 0.8442
BER @lamb = 100.0 is 0.1558
Accuracy @ lamb = 1000.0 is 0.84425
BER @lamb = 1000.0 is 0.15575
Accuracy @ lamb = 10000.0 is 0.84425
BER @lamb = 10000.0 is 0.15575
Accuracies = {1e-05: 0.68575, 0.0001: 0.6855, 0.001: 0.70075, 0.01: 0.8205, 0.1: 0.83625, 1.0: 0.84265, 10.0: 0.8442, 100.0: 0.8442, 1000.0: 0.84425, 10000.0: 0.84425}


In [50]:
test_set = []
prediction_file = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Played_classifierv3.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Played.txt"):
    if l.startswith("userID"):
        #header
        prediction_file.write(l)
        continue
    u,g = l.strip().split('-')
    val = {}
    val['userID'], val['gameID'] = u, g
    test_set.append(val)

model = linear_model.LogisticRegression(C=10000,class_weight='balanced')
model.fit(X_train, Y_train)
X_val = similarity_popularity_features(test_set)
Y_val = model.predict(X_val)
for i in range(len(Y_val)):
    u, g = test_set[i]['userID'], test_set[i]['gameID']
    pred = Y_val[i]
    prediction_file.write(u + "-" + g + "," + str(pred) + "\n")

prediction_file.close()
print("Done writing to the file!")

Done writing to the file!
